# Performance of Spark 2.0's Tungsten engine
### "whole-stage code generation"

This notebook demonstrates the power of whole-stage code generation, a technique that blends state-of-the-art from modern compilers and MPP databases. In order to compare the performance with Spark 1.6, we turn off whole-stage code generation in Spark 2.0, which would result in using a similar code path as in Spark 1.6.

To read the companion blog posts, click the following:
- https://databricks.com/blog/2016/05/11/spark-2-0-technical-preview-easier-faster-and-smarter.html
- https://databricks.com/blog/2016/05/23/apache-spark-as-a-compiler-joining-a-billion-rows-per-second-on-a-laptop.html

In [98]:
# Connect to Spark by creating a Spark session
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName("Benchmark")\
    .getOrCreate()

In [99]:
from time import time

def benchmark(name,MenthodToRun):
    startTime = time()
    MenthodToRun
    TotalTime = time() - startTime
    print("Time taken in {}: {} seconds".format(name, TotalTime))
    del startTime
    del TotalTime

# How fast can Spark 1.6 vs 2.0 sum up 1 billion numbers?

In [93]:
from time import time

#This config turns off whole stage code generation, effectively changing the execution path to be similar to Spark 1.6.
spark.conf.set("spark.sql.codegen.wholeStage", False)

#benchmark("Spark 1.6",spark.range(1000L * 1000 * 1000).selectExpr("sum(id)").show())

t0 = time()
spark.range(1000L * 1000 * 1000).selectExpr("sum(id)").show()
tt = time() - t0

print("Sum done in {} seconds without wholeStage CodeGen".format(round(tt,3)))

#This config turns on whole stage code generation, effectively changing the execution path to be similar to Spark 2.0.
spark.conf.set("spark.sql.codegen.wholeStage", True)

t0 = time()
spark.range(1000L * 1000 * 1000).selectExpr("sum(id)").show()
tt = time() - t0

print("Sum done in {} seconds with wholeStage CodeGen".format(round(tt,3)))

+------------------+
|           sum(id)|
+------------------+
|499999999500000000|
+------------------+

Sum done in 11.55 seconds without wholeStage CodeGen
+------------------+
|           sum(id)|
+------------------+
|499999999500000000|
+------------------+

Sum done in 0.431 seconds with wholeStage CodeGen



### Other primitive operations

Databricks has benchmarked the efficiency of other primitive operations under whole-stage code generation. The table below summarizes the result:

The way we benchmark is to to measure the cost per row, in nanoseconds.

Runtime: Intel Haswell i7 4960HQ 2.6GHz, HotSpot 1.8.0_60-b27, Mac OS X 10.11

|                       | Spark 1.6 | Spark 2.0 |
|:---------------------:|:---------:|:---------:|
|         filter        |   15 ns   |   1.1 ns  |
|     sum w/o group     |   14 ns   |   0.9 ns  |
|      sum w/ group     |   79 ns   |  10.7 ns  |
|       hash join       |   115 ns  |   4.0 ns  |
|  sort (8 bit entropy) |   620 ns  |   5.3 ns  |
| sort (64 bit entropy) |   620 ns  |   40 ns   |
|    sort-merge join    |   750 ns  |   700 ns  |


Again, to read the companion blog post, click here: https://databricks.com/blog/2016/05/11/spark-2-0-technical-preview-easier-faster-and-smarter.html

# How fast can Spark 1.6 vs 2.0 join 1 billion records?

In [103]:
spark.conf.set("spark.sql.codegen.wholeStage", False)

t0 = time()
spark.range(1000L * 1000 * 1000).join(spark.range(1000L).toDF("id"), "id").count()
tt = time() - t0

print("Join completed in {} seconds without wholeStage CodeGen".format(round(tt,3)))

spark.conf.set("spark.sql.codegen.wholeStage", True)

t0 = time()
spark.range(1000L * 1000 * 1000).join(spark.range(1000L).toDF("id"), "id").count()
tt = time() - t0

print("Join completed in {} seconds with wholeStage CodeGen".format(round(tt,3)))


Join completed in 34.575 seconds without wholeStage CodeGen
Join completed in 0.425 seconds with wholeStage CodeGen


# Understanding the execution plan

In [104]:
spark.range(1000).filter("id > 100").selectExpr("sum(id)").explain()

== Physical Plan ==
*HashAggregate(keys=[], functions=[sum(id#1078L)])
+- Exchange SinglePartition
   +- *HashAggregate(keys=[], functions=[partial_sum(id#1078L)])
      +- *Filter (id#1078L > 100)
         +- *Range (0, 1000, step=1, splits=4)


The explain function has been extended for whole-stage code generation. When an operator has a star around it (*), whole-stage code generation is enabled. In the following case, Range, Filter, and the two Aggregates are both running with whole-stage code generation. Exchange does not have whole-stage code generation because it is sending data across the network.

This query plan has two "stages" (divided by Exchange). In the first stage, three operators (Range, Filter, Aggregate) are collapsed into a single function. In the second stage, there is only a single operator (Aggregate).

Reference : Databricks Scala Code